<a href="https://colab.research.google.com/github/chidudals/CSCI4961_labs_projects/blob/main/tutorial_1/MLP_tutorial_quiz_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial 1: QUIZ**     


## Enable and test the GPU on Google Colab

In [1]:
import torch
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'Tesla T4'

##**Quiz 1:** You can find the code for logistic regression on FashionMNIST dataset  below. Please change the following two parameters and report the accuracy of the model on the 10000 test images:

(1-1) batch_size is 128 and learning rate is 0.0001 \
(1-2) batch_size is 128 and learning rate is 0.001 \
(1-3) batch_size is 128 and learning rate is 0.1




In [5]:
# ================================================================== #
#                           Logistic Regression                      #
# ================================================================== #
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Hyper-parameters
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 128 # default 100
learning_rate = 0.1

# fashionMNIST dataset (images and labels)
train_dataset = torchvision.datasets.FashionMNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/469], Loss: 0.8716
Epoch [1/5], Step [200/469], Loss: 0.6505
Epoch [1/5], Step [300/469], Loss: 0.5409
Epoch [1/5], Step [400/469], Loss: 0.4951
Epoch [2/5], Step [100/469], Loss: 0.5445
Epoch [2/5], Step [200/469], Loss: 0.5375
Epoch [2/5], Step [300/469], Loss: 0.5975
Epoch [2/5], Step [400/469], Loss: 0.4355
Epoch [3/5], Step [100/469], Loss: 0.3519
Epoch [3/5], Step [200/469], Loss: 0.5634
Epoch [3/5], Step [300/469], Loss: 0.3584
Epoch [3/5], Step [400/469], Loss: 0.5024
Epoch [4/5], Step [100/469], Loss: 0.4334
Epoch [4/5], Step [200/469], Loss: 0.3972
Epoch [4/5], Step [300/469], Loss: 0.4774
Epoch [4/5], Step [400/469], Loss: 0.4536
Epoch [5/5], Step [100/469], Loss: 0.4461
Epoch [5/5], Step [200/469], Loss: 0.6201
Epoch [5/5], Step [300/469], Loss: 0.4708
Epoch [5/5], Step [400/469], Loss: 0.5475
Accuracy of the model on the 10000 test images: 83.16000366210938 %


##**Quiz 2:** You can find the code for fully connected layer on FashionMNIST dataset  below. Please change the following two parameters and report the accuracy of the model on the 10000 test images:

(2-1) change hidden size to 64 with the Adam optimizer.\
(2-2) change hidden size to 1024 with the Adam optimizer.\
(2-3) change hidden size to 8192 with the Adam optimizer.

In [8]:
# ================================================================== #
#                      Feedforward Neural Network                    #
# ================================================================== #
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters

input_size = 784
hidden_size = 8192 # default: 500
num_classes = 10
num_epochs = 5
batch_size = 300
learning_rate = 0.001


# FashionMNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/200], Loss: 0.4455
Epoch [1/5], Step [200/200], Loss: 0.4287
Epoch [2/5], Step [100/200], Loss: 0.3999
Epoch [2/5], Step [200/200], Loss: 0.3375
Epoch [3/5], Step [100/200], Loss: 0.2937
Epoch [3/5], Step [200/200], Loss: 0.2610
Epoch [4/5], Step [100/200], Loss: 0.2692
Epoch [4/5], Step [200/200], Loss: 0.3389
Epoch [5/5], Step [100/200], Loss: 0.2540
Epoch [5/5], Step [200/200], Loss: 0.2828
Accuracy of the network on the 10000 test images: 88.06 %


(2-4) use SGD optimizer (keep other parameters as default)\
(2-5) use RMSprop optimizer (keep other parameters as default)\
Check the following optimizer list \
https://pytorch.org/docs/stable/optim.html

In [11]:
# ================================================================== #
#                      Feedforward Neural Network                    #
# ================================================================== #
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters

input_size = 784
hidden_size = 200
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


# FashionMNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.5493
Epoch [1/5], Step [200/600], Loss: 0.4247
Epoch [1/5], Step [300/600], Loss: 0.5873
Epoch [1/5], Step [400/600], Loss: 0.3733
Epoch [1/5], Step [500/600], Loss: 0.5582
Epoch [1/5], Step [600/600], Loss: 0.4135
Epoch [2/5], Step [100/600], Loss: 0.3881
Epoch [2/5], Step [200/600], Loss: 0.3269
Epoch [2/5], Step [300/600], Loss: 0.3350
Epoch [2/5], Step [400/600], Loss: 0.3550
Epoch [2/5], Step [500/600], Loss: 0.3377
Epoch [2/5], Step [600/600], Loss: 0.3217
Epoch [3/5], Step [100/600], Loss: 0.2740
Epoch [3/5], Step [200/600], Loss: 0.3000
Epoch [3/5], Step [300/600], Loss: 0.3672
Epoch [3/5], Step [400/600], Loss: 0.2546
Epoch [3/5], Step [500/600], Loss: 0.4617
Epoch [3/5], Step [600/600], Loss: 0.2654
Epoch [4/5], Step [100/600], Loss: 0.2065
Epoch [4/5], Step [200/600], Loss: 0.5072
Epoch [4/5], Step [300/600], Loss: 0.2444
Epoch [4/5], Step [400/600], Loss: 0.3582
Epoch [4/5], Step [500/600], Loss: 0.2016
Epoch [4/5], Step [600/600], Loss: